In [21]:
import numpy as np
import pandas as pd
import re
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from stop_words import get_stop_words

In [22]:
kickstarter = pd.read_csv("csv/ks-projects-201801.csv",sep=',')

In [23]:
kickstarter.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [24]:
len(kickstarter)

378661

In [25]:
kickstarter = kickstarter[0:10000]
kickstarter

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,1126959830,Short Bus for the GypsyCab (Canceled),Performance Art,Art,USD,2011-12-20,4000.0,2011-12-10 02:28:17,325.0,canceled,12,US,325.0,325.0,4000.00
24996,1126959928,Help to FIght Domestic Violence,Publishing,Publishing,USD,2015-12-15,2000.0,2015-11-30 18:44:54,0.0,undefined,0,"N,0""",NaN,0.0,2000.00
24997,112696733,There Are No Tangents: Part II - Re[membrane]i...,Dance,Dance,USD,2013-06-14,4000.0,2013-05-18 23:51:10,0.0,canceled,0,US,0.0,0.0,4000.00
24998,1126984813,A portrait of The Binghampton Neighborhood in ...,Photography,Photography,USD,2014-02-14,25000.0,2014-01-04 18:10:15,3291.0,failed,52,US,3291.0,3291.0,25000.00


In [26]:
kickstarter['category'] = kickstarter['category'].apply(lambda category: re.sub('[^A-Za-z]+', ' ', category.lower()))

In [27]:
kickstarter.isnull().sum()

ID                    0
name                  0
category              0
main_category         0
currency              0
deadline              0
goal                  0
launched              0
pledged               0
state                 0
backers               0
country               0
usd pledged         264
usd_pledged_real      0
usd_goal_real         0
dtype: int64

378661

In [28]:
clf = make_pipeline(
    TfidfVectorizer(stop_words=get_stop_words('en')),
    OneVsRestClassifier(SVC(kernel='linear', probability=True))
)

In [29]:
X = kickstarter['category']
Y = kickstarter['pledged'].astype('int')
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
len(X_train)

20000

In [30]:
clf = clf.fit(X=X_train, y=Y_train)